In [69]:
from nltk.translate.bleu_score import SmoothingFunction
from bert_score import score
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import evaluate
import json
import nltk
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from tqdm import tqdm
smoothie = SmoothingFunction().method4
rouge = evaluate.load('rouge')
messages_final = json.load(open('messages_final.json'))
messages_boh_original_final = json.load(open('messages_boh_original_final.json'))
messages_no_sw_original_final = json.load(open('messages_no_sw_original_final.json'))
test_data = json.load(open('test_data.json'))

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/csgrad/sunilruf/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
len(messages_final), len(messages_boh_original_final), len(messages_no_sw_original_final)

(100, 100, 100)

In [22]:
len(messages_final[0]), len(messages_boh_original_final[0]), len(messages_no_sw_original_final[0])

(30, 30, 30)

In [87]:
i = 1
bleu_2_final, bleu_2_boh_original_final, bleu_2_no_sw_original_final = [], [], []
bleu_4_final, bleu_4_boh_original_final, bleu_4_no_sw_original_final = [], [], []
bert_score_final, bert_score_boh_original_final, bert_score_no_sw_original_final = [], [], []
rouge_final, rouge_boh_original_final, rouge_no_sw_original_final = [], [], []
meteor_final, meteor_boh_original_final, meteor_no_sw_original_final = [], [], []

for j in tqdm(range(100)):
    
    for i in range(1, len(test_data[j]),2):
        #print(i)
        if messages_final[j][i]['content'] == '' or test_data[j][i]['content'] == '' or messages_boh_original_final[j][i]['content'] == '' or messages_no_sw_original_final[j][i]['content'] == '':
            continue
        #print(messages_final[j][i]['content'])
        
        bleu_2_final.append(sentence_bleu(messages_final[j][i]['content'], test_data[j][i]['content'], weights=(0.5,0.5), smoothing_function=smoothie))
        bleu_2_boh_original_final.append(sentence_bleu(messages_boh_original_final[j][i]['content'], test_data[j][i]['content'], weights=(0.5,0.5), smoothing_function=smoothie))
        bleu_2_no_sw_original_final.append(sentence_bleu(messages_no_sw_original_final[j][i]['content'], test_data[j][i]['content'], weights=(0.5,0.5), smoothing_function=smoothie))
        bleu_4_final.append(sentence_bleu(messages_final[j][i]['content'], test_data[j][i]['content'], weights=(0.25,0.25,0.25,0.25), smoothing_function=smoothie))
        bleu_4_boh_original_final.append(sentence_bleu(messages_boh_original_final[j][i]['content'], test_data[j][i]['content'], weights=(0.25,0.25,0.25,0.25), smoothing_function=smoothie))
        bleu_4_no_sw_original_final.append(sentence_bleu(messages_no_sw_original_final[j][i]['content'], test_data[j][i]['content'], weights=(0.25,0.25,0.25,0.25), smoothing_function=smoothie))
        bert_score_final.append(score([messages_final[j][i]['content']], [test_data[j][i]['content']], lang='en', model_type='bert-base-uncased'))
        bert_score_boh_original_final.append(score([messages_boh_original_final[j][i]['content']], [test_data[j][i]['content']], lang='en', model_type='bert-base-uncased'))
        bert_score_no_sw_original_final.append(score([messages_no_sw_original_final[j][i]['content']], [test_data[j][i]['content']], lang='en', model_type='bert-base-uncased'))
        rouge_final.append(rouge.compute(predictions=[messages_final[j][i]['content']], references=[test_data[j][i]['content']]))
        rouge_boh_original_final.append(rouge.compute(predictions=[messages_boh_original_final[j][i]['content']], references=[test_data[j][i]['content']]))
        rouge_no_sw_original_final.append(rouge.compute(predictions=[messages_no_sw_original_final[j][i]['content']], references=[test_data[j][i]['content']]))
        meteor_final.append((nltk.translate.meteor_score.meteor_score([word_tokenize(messages_final[j][i]['content'])], word_tokenize(test_data[j][i]['content']))))
        meteor_boh_original_final.append((nltk.translate.meteor_score.meteor_score([word_tokenize(messages_boh_original_final[j][i]['content'])], word_tokenize(test_data[j][i]['content']))))
        meteor_no_sw_original_final.append((nltk.translate.meteor_score.meteor_score([word_tokenize(messages_no_sw_original_final[j][i]['content'])], word_tokenize(test_data[j][i]['content']))))
        
        

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [21:47<00:00, 13.08s/it]


In [92]:
rougel_final = [i['rougeL'] for i in rouge_final]
rougel_boh_original_final = [i['rougeL'] for i in rouge_boh_original_final]
rougel_no_sw_original_final = [i['rougeL'] for i in rouge_no_sw_original_final]

In [98]:
bert_p_boh = [i[0].item() for i in bert_score_boh_original_final]
bert_r_boh = [i[1].item() for i in bert_score_boh_original_final]
bert_f1_boh = [i[2].item() for i in bert_score_boh_original_final]

bert_p_no_sw = [i[0].item() for i in bert_score_no_sw_original_final]
bert_r_no_sw = [i[1].item() for i in bert_score_no_sw_original_final]
bert_f1_no_sw = [i[2].item() for i in bert_score_no_sw_original_final]

bert_p = [i[0].item() for i in bert_score_final]
bert_r = [i[1].item() for i in bert_score_final]
bert_f1 = [i[2].item() for i in bert_score_final]

In [100]:
import statistics

print("Bleu 2 Final: ", statistics.mean(bleu_2_final))
print("Bleu 2 Boh Original Final: ", statistics.mean(bleu_2_boh_original_final))
print("Bleu 2 No Sw Original Final: ", statistics.mean(bleu_2_no_sw_original_final))
print("Bleu 4 Final: ", statistics.mean(bleu_4_final))
print("Bleu 4 Boh Original Final: ", statistics.mean(bleu_4_boh_original_final))
print("Bleu 4 No Sw Original Final: ", statistics.mean(bleu_4_no_sw_original_final))
print("Rouge L Final: ", statistics.mean(rougel_final))
print("Rouge L Boh Original Final: ", statistics.mean(rougel_boh_original_final))
print("Rouge L No Sw Original Final: ", statistics.mean(rougel_no_sw_original_final))
"""print("Bert Precision Final: ", statistics.mean(bert_p))
print("Bert Recall Final: ", statistics.mean(bert_r))
print("Bert F1 Final: ", statistics.mean(bert_f1))"""
print("Bert Precision Boh Original Final: ", statistics.mean(bert_p_boh))
print("Bert Recall Boh Original Final: ", statistics.mean(bert_r_boh))
print("Bert F1 Boh Original Final: ", statistics.mean(bert_f1_boh))
print("Bert Precision No Sw Original Final: ", statistics.mean(bert_p_no_sw))
print("Bert Recall No Sw Original Final: ", statistics.mean(bert_r_no_sw))
print("Bert F1 No Sw Original Final: ", statistics.mean(bert_f1_no_sw))
print("Meteor Final: ", statistics.mean(meteor_final))
print("Meteor Boh Original Final: ", statistics.mean(meteor_boh_original_final))
print("Meteor No Sw Original Final: ", statistics.mean(meteor_no_sw_original_final))


Bleu 2 Final:  0.023692008143055214
Bleu 2 Boh Original Final:  0.022857128164688597
Bleu 2 No Sw Original Final:  0.022651871236492992
Bleu 4 Final:  0.005390839336131438
Bleu 4 Boh Original Final:  0.005293997520894258
Bleu 4 No Sw Original Final:  0.005270120133785091
Rouge L Final:  0.29875519616293467
Rouge L Boh Original Final:  0.23433511035044557
Rouge L No Sw Original Final:  0.23055794902071927
Bert Precision Boh Original Final:  0.563170210429619
Bert Recall Boh Original Final:  0.5434744501494626
Bert F1 Boh Original Final:  0.5501792549036673
Bert Precision No Sw Original Final:  0.600833372299104
Bert Recall No Sw Original Final:  0.5502295829519827
Bert F1 No Sw Original Final:  0.5712984863028653
Meteor Final:  0.34693737737028274
Meteor Boh Original Final:  0.2812788387187525
Meteor No Sw Original Final:  0.32224831258546205


In [102]:
messages_boh = json.load(open('messages_boh_final.json'))
messages_sw = json.load(open('messages_no_sw_final.json'))
messages_final = json.load(open('messages_final.json'))

In [107]:
boh_length, sw_length, final_length = [], [], []
for j in tqdm(range(100)):
    for i in range(1,len(messages_final[j]),2):
        final_length.append(len(messages_final[j][i]['content']))
        sw_length.append(len(messages_sw[j][i]['content']))
        boh_length.append(len(messages_boh[j][i]['content']))

100%|██████████| 100/100 [00:00<00:00, 32615.12it/s]


In [108]:
print("actual length ", statistics.mean(final_length))
print("boh length ", statistics.mean(boh_length))
print("sw length ", statistics.mean(sw_length))

actual length  226.32758620689654
boh length  135.02909482758622
sw length  102.3728448275862
